### Generating embeddings using FaceNet Model

In [ ]:
!pip install facenet-pytorch

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch
import numpy as np
from PIL import Image

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

mtcnn = MTCNN(keep_all=True)

# load resnet from pretrained model 20180402-114759-vggface2.pt
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
# testing usage of model

embedding_path = "datasets/ffhq256_pp/train/embeddings/00000.npy"
image_path = "datasets/ffhq256/00000.png"

emb = np.load(embedding_path)


img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img = mtcnn(img)

img = img.reshape(1, 3, 160, 160)
emb2 = resnet(img).detach().numpy()

emb2-emb

In [ ]:
## generate and save embeddings for all images in the dataset
import numpy as np
dataset_path = 'datasets/ffhq256/'

import os
img_files = os.listdir(dataset_path)
img_files.sort()

start_index = 49053
end_index = len(img_files)

for i in range(start_index, end_index):
    img_path = dataset_path + img_files[i]
    img_copy_path = dataset_path + 'images/' + img_files[i]

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_cropped = mtcnn(img, save_path=img_copy_path)
    if img_cropped is None:
        print('No face detected in image: ', img_files[i])
        continue

    if img_cropped.shape[0] > 1:
        # remove the other faces
        for j in range(1, img_cropped.shape[0]):
            os.remove(img_copy_path.split('.')[0] + '_' + str(j+1) + '.png')
        
        img_cropped = img_cropped[0].reshape(1, 3, 160, 160)


    try:
        img_embedding = resnet(img_cropped)
    except:
        print('Error in processing image: ', img_files[i])
        # remove the image copy
        os.remove(img_copy_path)
        continue
    
    embedding = img_embedding.detach().numpy()

    file_name = dataset_path + 'embeddings/' + img_files[i].split('.')[0] + '.npy'
    np.save(file_name, embedding)


    if i % 1000 == 0:
        print('Processed ', i, ' images')

In [ ]:
i

In [ ]:
!mv datasets/ffhq256_new/embeddings datasets/ffhq256_new/train/

In [ ]:
# split intro train test
import os
import shutil

dataset_path = 'datasets/ffhq256/'
img_files = os.listdir(dataset_path + 'images/')
emb_files = os.listdir(dataset_path + 'embeddings/')
img_files.sort()
emb_files.sort()

train_path = dataset_path + 'train/'
test_path = dataset_path + 'test/'

if not os.path.exists(train_path):
    os.makedirs(train_path)

if not os.path.exists(test_path):
    os.makedirs(test_path)




In [ ]:
!mkdir datasets/ffhq256_new

In [ ]:
!mv /datasets/ffhq256/train /datasets/ffhq256_new/